In [6]:
import os
from IPython.display import Image
Image(filename="../input/picture/datachallenge.png", width= 900, height=500)

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
import datetime
import time
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
path = '../input/bank-customer-segmentation/bank_transactions.csv'
data = pd.read_csv(path)

- The dataset has 9 features 
- This dataset consists of 1 Million+ transaction by over 800K customers for a bank in India

In [9]:
data.head()

# Data Cleaning




In [10]:
data.info()

We dropped all null values as they were less than 0.5% 

In [11]:
data.dropna(axis=0,inplace = True)
data=data.reset_index(drop = True)

In [12]:
data['CustomerDOB'] = pd.to_datetime(data['CustomerDOB'], dayfirst=True)
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], dayfirst=True)

In [13]:
def fix_date(x):
    if (x.year > 2022) or (x.year < 1972):
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

In [14]:
data['CustomerDOB'] = data['CustomerDOB'].apply(fix_date)
data['TransactionDate'] = data['TransactionDate'].apply(fix_date)

We created a new feature "Age" from given dataset to help us building our insights later.

In [15]:
#data['TransactionDate'].sub(data['CustomerDOB'],axis = 0)
data['Age']=(data['TransactionDate'].values-data['CustomerDOB'].values)/365

In [16]:
def years(x):
    return x.days

In [17]:
data['Age']=data['Age'].apply(years)
mod=data['Age'].mode()
data['Age']=data['Age'].apply(lambda x:mod[0] if (x>120) else x)
data['Age']=data['Age'].apply(lambda x:mod[0] if (x<10) else x)

In [18]:
data.info()

In [19]:
data.describe()

In [20]:
numeric = ['CustAccountBalance','TransactionAmount (INR)','Age']

In [21]:
datatime = ['CustomerDOB','TransactionDate','TransactionTime']
categorical = ['CustGender','CustomerID','CustLocation']

In [22]:
data_below80=data[(data['Age']<80)]
data_above80=data[(data['Age']<100) & (data['Age']>80)]

In [23]:
data_above80.describe()

In [24]:
data_below80.describe()

In [25]:
plt.hist(np.log10(data_below80['CustAccountBalance']+0.001),log=True);
#plt.ylim([0,1000])

In [26]:
plt.hist(np.log10(data_above80['CustAccountBalance']+0.001),log=True);

In [27]:
sns.distplot(np.log10(data_above80['CustAccountBalance']+0.001))

In [28]:
sns.distplot(np.log10(data_below80['CustAccountBalance']+0.001),label='below')
sns.distplot(np.log10(data_above80['CustAccountBalance']+0.001),label='above')
sns.distplot(np.log10(data['CustAccountBalance']+0.001),label='all')
plt.legend();

In [29]:
sns.distplot(np.log10(data_below80['TransactionAmount (INR)']+0.001),label='below')
sns.distplot(np.log10(data_above80['TransactionAmount (INR)']+0.001),label='above')
sns.distplot(np.log10(data['TransactionAmount (INR)']+0.001),label='all');
plt.legend()

We found out the gender column has 3 categeries but one of these categeories has been repeated only once, So we removed it.

In [30]:
sns.histplot(data['CustGender']);

In [31]:
data['CustGender'].value_counts()

In [32]:
data = data[data["CustGender"]!='T']

In [33]:
data['CustGender'].value_counts().plot(kind='pie',autopct='%.2f')
plt.legend()
plt.show()

Here is a try to figure out the relation between customer account balance and transaction amount.

In [34]:
sns.scatterplot(x=np.log10(data[numeric[0]]),y=np.log10(data[numeric[1]]),
                alpha=0.002,hue=data["CustGender"],
                style=data["CustGender"],
                palette="deep");

Figuring out the distribution of transactions gender wise:

In [35]:
plt.hist(data[data["CustGender"]=='F'][numeric[1]],alpha=0.3,color=['red'],bins=range(0,2000,100),density=True);
plt.hist(data[data["CustGender"]=='M'][numeric[1]],alpha=0.5,bins=range(0,2000,100),density=True);

In [36]:
plt.boxplot(np.log10(data[numeric[0]]+0.001));

In [37]:
plt.boxplot(np.log10(data[numeric[1]]+0.001));

In [38]:
plt.boxplot((data[numeric[2]]));

In [39]:
sns.histplot(data['Age'])
plt.xlim([0,110]);
#plt.ylim([0,200])

It is clear that ages from 22 to 35 represent the majority of the data .

Inspecting outliers:

In [40]:
data.describe()

In [41]:
frame = {'more_3rd':[],
         'less_1st':[]}
more_3rd = []
less_1st = []
df=data
for i in data[numeric].columns:
    Q3 = data[numeric][i].quantile(0.75)
    Q1 = data[numeric][i].quantile(0.25)
    IQR= Q3-Q1
    more_3rd.append((data[numeric][i]>(Q3+1.5*IQR)).sum())  
    less_1st.append((data[numeric][i]<(Q1-1.5*IQR)).sum())
    df = df[df[numeric][i]<(Q3+1.5*IQR)]
    df = df[df[numeric][i]>(Q1-1.5*IQR)]
frame['more_3rd'] = more_3rd
frame['less_1st'] = less_1st
out=pd.DataFrame(frame, index = data[numeric].columns)
out

90% of customers are limited-income people

In [42]:
plt.hist(df['CustAccountBalance'],bins=range(0,150000,1000));
#plt.xticks(range(-4,9,1));
#plt.yscale('log');
plt.xticks(rotation  = 90);

Statistical aspect of TransactionAmount after data cleaning:

In [43]:
sns.boxplot(y=df['TransactionAmount (INR)'])

## EDA 

### Location based Analysis:

In [44]:
#import requests
#response = requests.get("https://shivammathur.com/countrycity/cities/India")

#print(response)
#city=response.json()
#cities=set(list(map(lambda x: x.lower(), city)))"
#location=set(data['CustLocation'].value_counts().keys())
#locations=set(list(map(lambda x: x.lower(), location)))
#inter=locations.intersection(cities)
#diff=locations.difference(cities)
#from geopy.geocoders import Nominatim

# initialize Nominatim API 
#geolocator = Nominatim(user_agent="geoapiExercises")
#location = geolocator.geocode("senapati district,India")
#c=0
#def geo_loc(inp):
 #   global c
    #print(inp)
  #  g=inp
   # c+=1
    #geolocator = Nominatim(user_agent=str(c))
    #location = geolocator.geocode(g+",India")
    #if location==None:
     #   return None
    #return location.latitude, location.longitude


#data['long_lat']=data['CustLocation'].apply(geo_loc)
#print(location.latitude, location.longitude)
#location = geolocator.reverse("33.76326745, -84.39511726814364")
#print(location.address)

In [45]:
plt.figure(figsize=(20,5))
plt.bar(data['CustLocation'][data["CustGender"]=='F'].value_counts()[:30].keys(),data['CustLocation'].value_counts()[:30],alpha=0.3,label="F")
plt.bar(data['CustLocation'][data["CustGender"]=='M'].value_counts()[:30].keys(),data['CustLocation'].value_counts()[:30],alpha=0.3,label="M")
plt.xticks(rotation = 90)
plt.legend()
plt.show()

In [46]:
sns.scatterplot(x=data[numeric[0]][data['CustLocation']=='MEERUT'],y=data[numeric[1]][data['CustLocation']=='MEERUT'],
                alpha=1,hue=data["CustGender"],
                style=data["CustGender"],
                palette="deep")

In [47]:
dens = data['CustLocation'].value_counts()[:10]

In [48]:
sns.histplot(data['Age'])

In [49]:
i=1
plt.figure(figsize=(20,20))
for city in dens.keys():   
    plt.subplot(4,3,i)
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.title("Age dist. in " + city)
    plt.hist(data[numeric[2]][data['CustLocation']==city],alpha = 1,label=city,bins=range(10,400,5),rwidth=0.5)
    #sns.histplot(df[numeric[2]][df['CustLocation']=='MUMBAI'],alpha=0.5,label="MUMBAI")
    i+=1
plt.show()

In [50]:
i=1
plt.figure(figsize=(20,20))
for city in dens.keys():   
    plt.subplot(4,3,i)
    plt.title("Transaction amount dist. in " + city)
    sns.histplot(data[numeric[1]][data['CustLocation']==city],alpha = 1,label=city)
    #sns.histplot(df[numeric[2]][df['CustLocation']=='MUMBAI'],alpha=0.5,label="MUMBAI")
    i+=1
plt.show()

### Gender based Analysis:

In [51]:
plt.hist(data[data["CustGender"]=='F'][numeric[2]],alpha=0.3,color=['red'],label='Female');
plt.hist(data[data["CustGender"]=='M'][numeric[2]],alpha=0.5,label='Male');
plt.title("Age dist. gender wise")
plt.xlabel("Age")
plt.ylabel("Count")
plt.legend();

**Feature Engineering**

In [52]:
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

In [53]:
data['TransactionDay'] = data['TransactionTime'].apply(time.ctime)
data['tr_hr']          = data['TransactionDay'].apply(lambda x : x[10:-4])
data['tr_year']        = data['TransactionDate'].apply(lambda x : x.year)
data['tr_month']       = data['TransactionDate'].apply(lambda x : x.month)
data['tr_day']         = data['TransactionDate'].apply(lambda x : x.day)
data['tr_weekday']     = data['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [54]:
hh=[int(datetime.datetime.fromtimestamp(i).strftime("%H")) for i in data.TransactionTime.values]
data['hr BY hr'] = hh

In [55]:
time = []
for i in hh:
    if i >=6 and i < 13:
        time.append("Morning")
    elif i >= 13 and i<=17:
        time.append("Afternoon")
    elif i > 17 and i <= 22:
        time.append("Evening")
    elif i > 22:
        time.append("Night")
    elif i < 6:
        time.append("Night")
data['tr_pr'] = time

In [56]:
enc_week={'Mon':0,'Tue':1,'Wed':2,'Thu':3,'Fri':4,'Sat':5,'Sun':6}
enc_per={'Morning':0,'Afternoon':1,'Evening':2,'Night':3}
enc_gender={'M':0,'F':1}

In [57]:
data['tr_weekday_enc']=data['tr_weekday'].apply(lambda x:enc_week[x])

In [58]:
data['tr_pr_enc']=data['tr_pr'].apply(lambda x:enc_per[x])

In [59]:
data['gender_enc']=data['CustGender'].apply(lambda x:enc_gender[x])

In [60]:
data.head()

In [61]:
data.info()

In [62]:
df_m=data[['CustAccountBalance','TransactionAmount (INR)','Age','tr_day','tr_month','tr_weekday_enc','tr_pr_enc','gender_enc']]

## Time based Analysis

- We splitted transaction data column into year , month and day to closely inspect each of these features
- We also divided day into 4 periods to isnpect activity of transaction in each period

### Day period activity

In [63]:
sns.barplot(data['tr_pr'].value_counts().keys(),data['tr_pr'].value_counts())
plt.title("count Vs period");

In [64]:
display(data[data['tr_pr']=='Morning']['TransactionAmount (INR)'].describe(),data[data['tr_pr']=='Night']['TransactionAmount (INR)'].describe())

In [65]:
data['Atm']=data['tr_pr']=='Night'

### Activity of each hour per day

In [66]:
plt.figure(figsize=(10,5))
sns.histplot(hh);
plt.xlabel('Day hours')
plt.title("Count vs Day hours")
plt.xticks(range(0,25,1));
plt.show()

### Activity of each day per month

In [67]:
plt.figure(figsize=(10,5))
plt.xlabel("months Day")
plt.axhline(y=34000,color='black');
sns.barplot(data['tr_day'].value_counts().keys(),data['tr_day'].value_counts());

In [68]:
plt.figure(figsize=(10,5))
sns.histplot(data[data['tr_month']==9]['tr_day']);
plt.xticks(range(1,32,1));
plt.title("dist. of Days through Sept.")
plt.show()

**first half** of the months has **more** **transactions** than **second half**

### Activity through months

In [69]:
sns.barplot(data['tr_month'].value_counts().keys(),data['tr_month'].value_counts());
plt.title('Count of months');

the data collected for 3 months only which is weird

### Activity through week

In [70]:
sns.barplot(data['tr_weekday'].value_counts().keys(),data['tr_weekday'].value_counts());

In [71]:
data['tr_ratio'] = data['TransactionAmount (INR)'] *100 / (data['CustAccountBalance']+0.01)

In [72]:
data['deposit']=0

In [73]:
data['deposit']=data['tr_ratio']>100
idx=data[(data['tr_pr']=='Night') &(data['TransactionAmount (INR)']>30000)].index
data['deposit'].iloc[idx]=True
idx2=data[(data['TransactionAmount (INR)']<100) & (data['tr_pr']=='Night')].index
data['deposit'].iloc[idx2]=True

In [74]:
data['deposit'].value_counts()

In [75]:
data[data['deposit']==1]['tr_pr'].value_counts().plot(kind='pie',autopct='%.2f');

In [76]:
data[data['tr_pr']=='Night']['deposit'].value_counts().plot(kind='pie',autopct='%.2f');

### Ratio between TransactionAmount (INR) to CustAccountBalance

In [77]:
sns.histplot(np.log10(data['tr_ratio']+0.001))
plt.title("tr_ratio")
plt.axvline(x=0.45,color='red');

most of the transactions are about 3% of the account balance

In [78]:
new_customer  = data[data['CustAccountBalance']==0]

We created a new data frame with the new customers to inspect them more carefully
hint : we figured that these customers are new from their balance account

In [79]:
new_customer.head()

In [80]:
plt.figure(figsize=(10,5))
plt.hist(new_customer['tr_day'],bins = range(0,32,1),rwidth=0.25);
plt.xticks(range(0,32,1));
plt.axhline(y=50,color='red');
plt.title("new customers for every day")
plt.show()

It is clear that through first half month there are more new customers than second one. 

We created a data frame by using group by to perform customer-wise analysis but the data wasn't sufficient

In [81]:
data['CustomerID'].value_counts()

In [82]:
costumers = data.groupby('CustomerID').agg({'TransactionAmount (INR)':'sum',
                                          'CustAccountBalance':['min','max'],
                                          'tr_ratio':['min' , 'max'],
                                          'Age':'first'}).reset_index()

In [83]:
costumers

In [84]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_m[['CustAccountBalance', 'TransactionAmount (INR)', 'Age']])
scaled_df=pd.DataFrame(scaled_data,columns=['CustAccountBalance', 'TransactionAmount (INR)', 'Age'])
scaled_df.head()

In [85]:
scaled_df[['tr_day','tr_month', 'tr_weekday_enc', 'tr_pr_enc', 'gender_enc']]=df_m[['tr_day','tr_month', 'tr_weekday_enc', 'tr_pr_enc', 'gender_enc']]
scaled_df.dropna(inplace=True)
scaled_df.head()

In [86]:
scaled_df.info()

In [87]:
scaled_df.describe()

#  Modelling 

In [88]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12)).fit(scaled_df)
visualizer.show()

In [89]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(scaled_df)
kmeans.labels_

In [90]:
out_put = kmeans.fit_predict(scaled_df)
Counter(kmeans.labels_)

In [91]:
plt.scatter(scaled_df.iloc[out_put == 0, 0], scaled_df.iloc[out_put == 0, 1],alpha=0.2, s = 50, c = 'red',label='A')
plt.scatter(scaled_df.iloc[out_put == 1, 0], scaled_df.iloc[out_put == 1, 1],alpha=0.2, s = 50, c = 'blue',label='B')
plt.scatter(scaled_df.iloc[out_put == 2, 0], scaled_df.iloc[out_put == 2, 1],alpha=0.2,s = 50, c = 'green',label='C')

# Plotting the centroids of the clusters
plt.scatter(kmeans.cluster_centers_[:,2], kmeans.cluster_centers_[:,3], s = 50, c = 'yellow', label = 'Centroids')

plt.legend()

In [92]:
plt.scatter(scaled_df.iloc[out_put == 0, 0], scaled_df.iloc[out_put == 0, 2],alpha=0.2, s = 50, c = 'red',label='A')
plt.scatter(scaled_df.iloc[out_put == 1, 0], scaled_df.iloc[out_put == 1, 2],alpha=0.2, s = 50, c = 'blue',label='B')
plt.scatter(scaled_df.iloc[out_put == 2, 0], scaled_df.iloc[out_put == 2, 2],alpha=0.2,s = 50, c = 'green',label='C')

# Plotting the centroids of the clusters
plt.scatter(kmeans.cluster_centers_[:,2], kmeans.cluster_centers_[:,3], s = 50, c = 'yellow', label = 'Centroids')

plt.legend()

In [93]:
plt.scatter(scaled_df.iloc[out_put == 0, 5], scaled_df.iloc[out_put == 0, 1],alpha=0.2, s = 50, c = 'red',label='A')
plt.scatter(scaled_df.iloc[out_put == 1, 5], scaled_df.iloc[out_put == 1, 1],alpha=0.2, s = 50, c = 'blue',label='B')
plt.scatter(scaled_df.iloc[out_put == 2, 5], scaled_df.iloc[out_put == 2, 1],alpha=0.2,s = 50, c = 'green',label='C')

# Plotting the centroids of the clusters
plt.scatter(kmeans.cluster_centers_[:,2], kmeans.cluster_centers_[:,3], s = 50, c = 'yellow', label = 'Centroids')

plt.legend()